<a href="https://colab.research.google.com/github/jayarnim/M0-RecommenderSystem/blob/main/DeepLearning/2_FM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# desc.

## sgd

- 손실 함수

$$
L= MSE + \lambda_{w}\sum_{i=1}^{n}{w_{i}^{2}} + \lambda_{v}\sum_{i=1}^{n}\sum_{j=1}^{k}{v_{i,j}^{2}}
$$

- 확률적 경사하강법

$$
\theta = \theta - \eta \frac{\partial L}{\partial \theta}
$$

- 편향 $\beta$ 갱신 방향

$$
\frac{\partial L}{\partial \beta}
= 2(\hat{y}-y)
$$

- $i$ 번째 특성에 대한 가중치 $w_{i}$ 갱신 방향

$$
\frac{\partial L}{\partial w_{i}}
= 2(\hat{y}-y)x_{i}+2\lambda_{w}w_{i}
$$

- $i$ 번째 특성에 대한 잠재요인 벡터 $\overrightarrow{v}_{i}$ 의 $j$ 번째 요소 갱신 방향

$$
\frac{\partial L}{\partial v_{i,j}}
= 2(\hat{y}-y)(\sum_{i=1}^{n}v_{i,j}x_{k}x_{i}-v_{k,j}x_{k}^2)+2\lambda_{v}v_{i,j}
$$

## predict

- `linear_terms`
$$
\beta + \sum_{i=1}^{n}{w_{i}x_{i}}
$$

- `interactions`

$$
\frac{1}{2}\sum_{i}\sum_{j\ne i}{\overrightarrow{v}_{i}^{T}\overrightarrow{v}_{j}x_{i}x_{j}}
$$

## compute_rmse

$$\begin{aligned}
\text{error} \in \text{errors}
&= (y - \hat{y})^2\\
&= \Big[y - \big\{\beta + \sum_{i=1}^{n}{w_{i}x_{i}} + \frac{1}{2}\sum_{i}\sum_{j\ne i}{\overrightarrow{v}_{i}^{T}\overrightarrow{v}_{j}x_{i}x_{j}}\big\}\Big]^2
\end{aligned}$$

In [ ]:
class FM():
    def __init__(self, n_factors, learning_rate, reg_w, reg_v, n_iterations):
        """
        Arguments
        n_factors       : 잠재요인 수
        learning_rate   : 학습률
        reg_w           : 가중치 행렬 정규화 강도
        reg_v           : 잠재요인 행렬 정규화 강도
        n_iterations    : 훈련 횟수
        """
        self.n_factors = n_factors
        self.learning_rate = learning_rate
        self.reg_w = reg_w
        self.reg_v = reg_v
        self.n_iterations = n_iterations


    def fit(self, X, y):
        n_samples, n_features = X.shape

        # 편향, 가중치, 잠재요인 초기화
        self.b = 0
        self.W = np.zeros(n_features)
        self.V = np.random.normal(scale=1./self.n_factors, size=(n_features, self.n_factors))

        training_process = []

        for iteration in range(self.n_iterations):
            np.random.shuffle(X)
            self.sgd(X, y)
            rmse = self.compute_rmse(X, y)
            training_process.append((i, rmse))
            print(f"Iteration: {iteration+1}, RMSE: {rmse}")

        return training_process


    def sgd(self, X, y):
        n_samples, n_features = X.shape

        for i in range(n_sample):
            # 예측 오차 계산
            error = self.predict(X[i]) - y[i]

            # 편향 갱신
            self.b -= self.learning_rate * error
            # 가중치 갱신
            self.W -= self.learning_rate * (error * X[i] + 2 * self.reg_w * self.W)
            # 잠재요인 갱신
            for factor in range(self.n_factors):
                v_grad = error * (self.X[i] * self.V[:, f] - np.dot(self.X[i], self.X[i] * self.V[:, factor]))
                self.V[:, factor] -= self.learning_rate * (v_grad + 2 * self.reg_v * self.V[:, factor])


    def predict(self, X):
        linear_terms = self.b + np.dot(X, self.W)
        interactions = 0.5 * np.sum(np.power(np.dot(X, self.V), 2) - np.dot(np.power(X, 2), np.power(self.V, 2)))

        return linear_terms + interactions


    def compute_rmse(self, X, y):
        errors = []
        for i in range(X.shape[0]):
            pred = self.predict(X[i])
            errors.append((pred - y[i]) ** 2)

        return np.sqrt(np.mean(errors))